# Imports

In [ ]:
import pandas as pd
import spacy

In [ ]:
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 43.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# Chargement des données

In [ ]:
# Création d'un dossier appelé data
!mkdir data

# Téléchargement du fichier contenant les fanfictions et leurs métadonnées
!wget -P data https://gitlab.unistra.fr/paulinemoreau/memoire/-/raw/main/data/vrac/pretraitements/valide/fanfics_perso.csv

--2025-03-19 18:57:35--  https://gitlab.unistra.fr/paulinemoreau/memoire/-/raw/main/data/vrac/pretraitements/valide/fanfics_perso.csv
Resolving gitlab.unistra.fr (gitlab.unistra.fr)... 130.79.254.48
Connecting to gitlab.unistra.fr (gitlab.unistra.fr)|130.79.254.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 289746887 (276M) [text/plain]
Saving to: ‘data/fanfics_perso.csv’

fanfics_perso.csv   100%[===================>] 276.32M  23.8MB/s    in 13s     

2025-03-19 18:57:48 (22.0 MB/s) - ‘data/fanfics_perso.csv’ saved [289746887/289746887]



In [ ]:
df_fanfic = pd.read_csv('data/fanfics_perso.csv')

In [ ]:
df_fanfic.head()

,work_id,tag,title,author,rating,category,fandom,relationship,character,additional tags,...,kudos,bookmarks,hits,all_kudos,all_bookmarks,body,body_clean,nb_caracteres,characters_clean,body_no_chara
0,63563206,Fluff,Nuit etoile,['Lady_Meg_666'],General Audiences,M/M,Harry Potter - J. K. Rowling,Sirius Black/Remus Lupin,"Sirius Black, Remus Lupin","Established Relationship, Fluff, Domestic Fluf...",...,NaN,NaN,14,[],[],"<< Regarde, murmure Sirius en montrant le ciel...","<< Regarde, murmure Sirius en montrant le ciel...",1413,"['Remus', 'Black', 'Sirius', 'Lupin']","<< Regarde, murmure en montrant le ciel de mi..."
1,63528274,Fluff,Je le deteste,['wilk'],Not Rated,M/M,Hawaii Five-0 (2010),"Steve McGarrett/Danny ""Danno"" Williams","Danny ""Danno"" Williams, Steve McGarrett",Fluff,...,5.0,NaN,59,"['MommaJ', 'WhiteShiro']",[],Danny deteste vraiment Steve : a peine deux jo...,Danny deteste vraiment Steve : a peine deux jo...,600,"['Steve', 'Williams', 'Danny', 'McGarrett', 'D...",deteste vraiment : a peine deux jours apres ...
2,63513724,Fluff,Camomille,['MyLovelyKitchenSinkDrama'],General Audiences,M/M,Guns N' Roses,Axl Rose/Izzy Stradlin,"Izzy Stradlin, Axl Rose","Domestic Fluff, Self-Indulgent, Trans Male Cha...",...,5.0,NaN,33,"['AllenKune', 'Sawcha', 'Azryel']",[],Izzy n'entend aucun bruit venir de l'appart qu...,Izzy n'entend aucun bruit venir de l'appart qu...,1742,"['Rose', 'Axl', 'Stradlin', 'Izzy']",n'entend aucun bruit venir de l'appart quand ...
3,63507442,Fluff,Le Sel sur ses Levres,['Kayukiya'],Explicit,M/M,Captive Prince - C. S. Pacat,Damen/Laurent (Captive Prince),"Damen (Captive Prince), Laurent (Captive Prince)","POV Damen (Captive Prince), Caring Damen (Capt...",...,6.0,NaN,88,"['emysterieuse', 'Shteakum', 'Azulina33']",[],\n\nLa jument a la superbe robe isabelle se d...,La jument a la superbe robe isabelle se dressa...,35140,"['Laurent', 'Damen']",La jument a la superbe robe isabelle se dressa...
4,63508948,Fluff,L'heureux destinataire,['Lady_Meg_666'],General Audiences,M/M,Teen Wolf (TV),Liam Dunbar/Theo Raeken,"Liam Dunbar, Theo Raeken","Letter, love letter, Romantic Fluff",...,1.0,NaN,41,['Myauada'],[],"""Je ne sais pas vraiment quoi faire. Depuis ma...","""Je ne sais pas vraiment quoi faire. Depuis ma...",2412,"['Theo', 'Liam', 'Raeken', 'Dunbar']","""Je ne sais pas vraiment quoi faire. Depuis ma..."


# Ajout de colonnes tokenisées

### Fonction de tokenisation

In [ ]:
spacy_pipeline = spacy.load("fr_core_news_sm", disable=["parser", "ner", "lemmatizer"])

# Fonction de tokenisation sans mots vides
def split_into_tokens_wosw_spacy(text):
    doc = spacy_pipeline(text)
    return ' '.join([t.text for t in doc if not t.is_stop])

### Tokenisation du body nettoyé avec noms des personnages

Je rajoute une colonne 'tokenised_body' contenant les textes des fanfictions (avec noms des personnages) tokenisés en appliquant la fonction 'split_into_tokens_wosw_spacy' à la colonne 'body_clean".

In [ ]:
df_fanfic['tokenised_body'] = df_fanfic['body_clean'].apply(split_into_tokens_wosw_spacy)

In [ ]:
df_fanfic.head()

,work_id,tag,title,author,rating,category,fandom,relationship,character,additional tags,...,bookmarks,hits,all_kudos,all_bookmarks,body,body_clean,nb_caracteres,characters_clean,body_no_chara,tokenised_body
0,63563206,Fluff,Nuit etoile,['Lady_Meg_666'],General Audiences,M/M,Harry Potter - J. K. Rowling,Sirius Black/Remus Lupin,"Sirius Black, Remus Lupin","Established Relationship, Fluff, Domestic Fluf...",...,NaN,14,[],[],"<< Regarde, murmure Sirius en montrant le ciel...","<< Regarde, murmure Sirius en montrant le ciel...",1413,"['Remus', 'Black', 'Sirius', 'Lupin']","<< Regarde, murmure en montrant le ciel de mi...","< < Regarde , murmure Sirius montrant ciel min..."
1,63528274,Fluff,Je le deteste,['wilk'],Not Rated,M/M,Hawaii Five-0 (2010),"Steve McGarrett/Danny ""Danno"" Williams","Danny ""Danno"" Williams, Steve McGarrett",Fluff,...,NaN,59,"['MommaJ', 'WhiteShiro']",[],Danny deteste vraiment Steve : a peine deux jo...,Danny deteste vraiment Steve : a peine deux jo...,600,"['Steve', 'Williams', 'Danny', 'McGarrett', 'D...",deteste vraiment : a peine deux jours apres ...,Danny deteste vraiment Steve : peine jours ren...
2,63513724,Fluff,Camomille,['MyLovelyKitchenSinkDrama'],General Audiences,M/M,Guns N' Roses,Axl Rose/Izzy Stradlin,"Izzy Stradlin, Axl Rose","Domestic Fluff, Self-Indulgent, Trans Male Cha...",...,NaN,33,"['AllenKune', 'Sawcha', 'Azryel']",[],Izzy n'entend aucun bruit venir de l'appart qu...,Izzy n'entend aucun bruit venir de l'appart qu...,1742,"['Rose', 'Axl', 'Stradlin', 'Izzy']",n'entend aucun bruit venir de l'appart quand ...,Izzy entend aucun bruit venir appart tourne cl...
3,63507442,Fluff,Le Sel sur ses Levres,['Kayukiya'],Explicit,M/M,Captive Prince - C. S. Pacat,Damen/Laurent (Captive Prince),"Damen (Captive Prince), Laurent (Captive Prince)","POV Damen (Captive Prince), Caring Damen (Capt...",...,NaN,88,"['emysterieuse', 'Shteakum', 'Azulina33']",[],\n\nLa jument a la superbe robe isabelle se d...,La jument a la superbe robe isabelle se dressa...,35140,"['Laurent', 'Damen']",La jument a la superbe robe isabelle se dressa...,jument superbe robe isabelle dressait fieremen...
4,63508948,Fluff,L'heureux destinataire,['Lady_Meg_666'],General Audiences,M/M,Teen Wolf (TV),Liam Dunbar/Theo Raeken,"Liam Dunbar, Theo Raeken","Letter, love letter, Romantic Fluff",...,NaN,41,['Myauada'],[],"""Je ne sais pas vraiment quoi faire. Depuis ma...","""Je ne sais pas vraiment quoi faire. Depuis ma...",2412,"['Theo', 'Liam', 'Raeken', 'Dunbar']","""Je ne sais pas vraiment quoi faire. Depuis ma...",""" sais vraiment faire . , coeur . oublier voud..."


### Tokenisation du body nettoyé sans noms des personnages

Je fais de même pour les textes où les noms de personnages ont été supprimés pour obtenir les textes tokenisés dans la colonne 'tokenised_no_chara'.

In [ ]:
df_fanfic['tokenised_no_chara'] = df_fanfic['body_no_chara'].apply(split_into_tokens_wosw_spacy)

In [ ]:
df_fanfic.head()

,work_id,tag,title,author,rating,category,fandom,relationship,character,additional tags,...,hits,all_kudos,all_bookmarks,body,body_clean,nb_caracteres,characters_clean,body_no_chara,tokenised_body,tokenised_no_chara
0,63563206,Fluff,Nuit etoile,['Lady_Meg_666'],General Audiences,M/M,Harry Potter - J. K. Rowling,Sirius Black/Remus Lupin,"Sirius Black, Remus Lupin","Established Relationship, Fluff, Domestic Fluf...",...,14,[],[],"<< Regarde, murmure Sirius en montrant le ciel...","<< Regarde, murmure Sirius en montrant le ciel...",1413,"['Remus', 'Black', 'Sirius', 'Lupin']","<< Regarde, murmure en montrant le ciel de mi...","< < Regarde , murmure Sirius montrant ciel min...","< < Regarde , murmure montrant ciel minuit p..."
1,63528274,Fluff,Je le deteste,['wilk'],Not Rated,M/M,Hawaii Five-0 (2010),"Steve McGarrett/Danny ""Danno"" Williams","Danny ""Danno"" Williams, Steve McGarrett",Fluff,...,59,"['MommaJ', 'WhiteShiro']",[],Danny deteste vraiment Steve : a peine deux jo...,Danny deteste vraiment Steve : a peine deux jo...,600,"['Steve', 'Williams', 'Danny', 'McGarrett', 'D...",deteste vraiment : a peine deux jours apres ...,Danny deteste vraiment Steve : peine jours ren...,"deteste vraiment : peine jours rencontre ,..."
2,63513724,Fluff,Camomille,['MyLovelyKitchenSinkDrama'],General Audiences,M/M,Guns N' Roses,Axl Rose/Izzy Stradlin,"Izzy Stradlin, Axl Rose","Domestic Fluff, Self-Indulgent, Trans Male Cha...",...,33,"['AllenKune', 'Sawcha', 'Azryel']",[],Izzy n'entend aucun bruit venir de l'appart qu...,Izzy n'entend aucun bruit venir de l'appart qu...,1742,"['Rose', 'Axl', 'Stradlin', 'Izzy']",n'entend aucun bruit venir de l'appart quand ...,Izzy entend aucun bruit venir appart tourne cl...,entend aucun bruit venir appart tourne clef ...
3,63507442,Fluff,Le Sel sur ses Levres,['Kayukiya'],Explicit,M/M,Captive Prince - C. S. Pacat,Damen/Laurent (Captive Prince),"Damen (Captive Prince), Laurent (Captive Prince)","POV Damen (Captive Prince), Caring Damen (Capt...",...,88,"['emysterieuse', 'Shteakum', 'Azulina33']",[],\n\nLa jument a la superbe robe isabelle se d...,La jument a la superbe robe isabelle se dressa...,35140,"['Laurent', 'Damen']",La jument a la superbe robe isabelle se dressa...,jument superbe robe isabelle dressait fieremen...,jument superbe robe isabelle dressait fieremen...
4,63508948,Fluff,L'heureux destinataire,['Lady_Meg_666'],General Audiences,M/M,Teen Wolf (TV),Liam Dunbar/Theo Raeken,"Liam Dunbar, Theo Raeken","Letter, love letter, Romantic Fluff",...,41,['Myauada'],[],"""Je ne sais pas vraiment quoi faire. Depuis ma...","""Je ne sais pas vraiment quoi faire. Depuis ma...",2412,"['Theo', 'Liam', 'Raeken', 'Dunbar']","""Je ne sais pas vraiment quoi faire. Depuis ma...",""" sais vraiment faire . , coeur . oublier voud...",""" sais vraiment faire . , coeur . oublier voud..."


In [ ]:
# Enregistrement
df_fanfic.to_csv('data/fanfics_tokenised.csv', index=False)